In [ ]:
#import data
data <- read.csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQhIE8Ii658C4Ex4BXo2kKfzZHLh3Bt84UqDUXL7wviqmIRN98RzvW9pX7jTywwzuwcvbElB4y31yAW/pub?gid=1950240221&single=true&output=csv")

head(data)

hyd_data <- subset(data, City == "Hyderabad")
dim(hyd_data)


#get rent stats
rent <- data$Rent

mean_rent <- mean(rent)
median_rent <- median(rent)
min_rent <- min(rent)
max_rent <- max(rent)

cat("Mean rent: ", mean_rent, "\n")
cat("Median rent: ", median_rent, "\n")
cat("Minimum rent: ", min_rent, "\n")
cat("Maximum rent: ", max_rent, "\n")

#get info on point of contact 
var_freq <- table(data$Point.of.Contact)
print(var_freq)


# Create dummy variable 
data$contact_dummy <- as.numeric(ifelse(data$Point.of.Contact == "Contact Owner", 0, 1))

# Remove observation with Contact Builder value
data <- subset(data, Point.of.Contact != "Contact Builder")

# Calculate the mean rent for the "Contact Owner" and "Contact Agent" groups
mean_rent_owner <- mean(data$Rent[data$contact_dummy == 0])
mean_rent_agent <- mean(data$Rent[data$contact_dummy == 1])


cat("Mean rent for Contact Owner: ", mean_rent_owner, "\n")
cat("Mean rent for Contact Agent: ", mean_rent_agent, "\n")
cat("If renting with agent one would pay on average", mean_rent_agent - mean_rent_owner,"more", "\n")


#Create new variable based on Furnishing.Status and convert to numeric

data$furnishing_dummy <- ifelse(data$Furnishing.Status == "Furnished", 2,ifelse(data$Furnishing.Status == "Unfurnished",0,1))
data$furnishing_dummy <- as.numeric(data$furnishing_dummy)



In [ ]:
#analysis 
set.seed(130)

#PART_1_______________________________ 

#create a new data frame as a combination of 4 variables 
X <- cbind(data$BHK, data$Size, data$furnishing_dummy, data$Bathroom)

#running regression 1
lm1 <- lm(data$Rent ~ data$Point.of.Contact + X)
summary(lm1)



#PART_2_______________________________ 

install.packages("rgenoud")
install.packages('Matching')
library(rgenoud)
library(Matching)

# Set seed for replicability
set.seed(130)

#generating weights 
gm_output <- GenMatch(estimand = "ATT",Tr = data$contact_dummy, X = X, pop.size = 100, max.generations=10, wait.generations=10, caliper = 0.05)

#matching with the weights obtianed 
mout <- Match(Tr = data$contact_dummy, X = X, Weight.matrix = gm_output,caliper = 0.05)

mb <- MatchBalance(
    data$contact_dummy ~ X,
    match.out=mout, nboots=500)


#PART_3_______________________________ 

#match considering outcome variable 
mout2 <- Match(Tr = data$contact_dummy, X = X,Y = data$Rent, Weight.matrix = gm_output,caliper = 0.05)

mb <- MatchBalance(
    data$contact_dummy ~ data$Rent,
    match.out=mout2, nboots=500)

nrow(matched_data)
nrow(data)ˇ


#PART_4_______________________________ 

# row numbers for matched units
treated_rows <- mout2$index.treated
control_rows <- mout2$index.control

# matched dataset
matched_data <- data[c(treated_rows, control_rows), ]

# Create unit weights
weightslm2 <- c(mout2$weights, mout2$weights)

X_matched <- cbind(matched_data$BHK, matched_data$Size, matched_data$furnishing_dummy, matched_data$Bathroom)

lm2 <- lm(matched_data$Rent ~ matched_data$contact_dummy + matched_data$BHK+ matched_data$Size+ matched_data$furnishing_dummy+ matched_data$Bathroom, data = matched_data, weights = weightslm2)
summary(lm2)


#PART_4_OPTIONAL______________________________ 

install.packages("quantreg")
library(quantreg)


#fit quantile regression

lm_q1 <- rq(matched_data$Rent~matched_data$contact_dummy+ matched_data$BHK+ matched_data$Size+matched_data$furnishing_dummy+ matched_data$Bathroom, tau = seq(0.025,0.975,by=0.1))
summary(lm_q1)
plot(lm_q1)

lm_q2 <- rq(data$Rent~data$contact_dummy+data$BHK+ data$Size+ data$furnishing_dummy+ data$Bathroom, tau = seq(0.025,0.975,by=0.1))
summary(lm_q2)
plot(lm_q2)


#PART_5_______________________________ 


install.packages("sensemakr")
library(sensemakr)

# Run sensitivity analysis
sensitivity <- sensemakr(model = lm2, treatment = 'matched_data$contact_dummy', benchmark_covariates = 'matched_data$Size', kd = 1:3)

plot(sensitivity)